In [1]:
!pip install --upgrade google-auth 

     |████████████████████████████████| 136 kB 1.6 MB/s eta 0:00:01
  Attempting uninstall: google-auth
    Found existing installation: google-auth 1.26.0
    Uninstalling google-auth-1.26.0:
      Successfully uninstalled google-auth-1.26.0


In [2]:
!pip install --upgrade absl-py

     |████████████████████████████████| 129 kB 1.9 MB/s eta 0:00:01
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.11.0
    Uninstalling absl-py-0.11.0:
      Successfully uninstalled absl-py-0.11.0


In [3]:
!pip install --upgrade protobuf

     |████████████████████████████████| 1.0 MB 1.9 MB/s eta 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.14.0
    Uninstalling protobuf-3.14.0:
      Successfully uninstalled protobuf-3.14.0


In [4]:
!pip install --upgrade -q tensorflow

In [5]:
# A dependency of the preprocessing for BERT inputs
!pip install -q tensorflow-text

In [6]:
!pip install -q tf-models-official

------------------------------------

In [7]:
import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub

In [13]:
tokenizer = hub.load(
    'https://code.aliyun.com/qhduan/bert_v4/raw/500019068f2c715d4b344c3e2216cef280a7f800/bert_tokenizer_chinese.tar.gz'
)

In [14]:
tokenizer

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7f4cd05669a0>

In [12]:
albert = hub.load(
    'https://code.aliyun.com/qhduan/bert_v4/raw/500019068f2c715d4b344c3e2216cef280a7f800/albert_tiny.tar.gz'
)

In [15]:
out = albert(tokenizer(['你好']))

In [16]:
out

{'pooled_output': <tf.Tensor: shape=(1, 312), dtype=float32, numpy=
 array([[ 9.97694671e-01, -8.06971908e-01,  5.54048657e-01,
          6.82475686e-01, -5.85427761e-01, -4.75501448e-01,
         -1.65914804e-01,  3.15567590e-02,  8.45138013e-01,
         -8.01114380e-01, -9.99356210e-01, -3.09111893e-01,
          9.99294996e-01, -9.22129214e-01,  9.92580652e-01,
          9.99752402e-01,  9.58445728e-01,  9.99814987e-01,
          9.99970496e-01,  4.10672754e-01, -1.31741986e-01,
          9.97253299e-01, -9.60790634e-01, -9.97110248e-01,
          5.04275501e-01,  2.64063597e-01, -9.54064131e-01,
          9.86726403e-01, -9.99091148e-01,  8.28569770e-01,
          9.83543754e-01, -1.33558795e-01,  9.98841763e-01,
          9.67339694e-01,  9.98867333e-01,  9.16883111e-01,
         -9.90573108e-01, -2.96226870e-02, -9.97980237e-01,
          2.11710617e-01,  1.94977745e-01, -5.33369184e-01,
          1.97668418e-01, -3.25063467e-01, -3.42272729e-01,
          9.95688736e-01,  2.400

In [17]:
#assert out['sequence_output'].shape == (1, 2, 312)

### BERT 分類模型

In [18]:
x = [
    ['天氣好'],
    ['天氣差'],
    ['下雨'],
    ['晴天'],
    ['雨'],
    ['晴'],
]
y = [
    1, 0, 0, 1, 0, 1
]

In [19]:
x,y

([['天氣好'], ['天氣差'], ['下雨'], ['晴天'], ['雨'], ['晴']], [1, 0, 0, 1, 0, 1])

In [20]:
tx = tf.constant(x)
ty = tf.constant(tf.keras.utils.to_categorical(y, 2))

In [21]:
tx,ty

(<tf.Tensor: shape=(6, 1), dtype=string, numpy=
 array([[b'\xe5\xa4\xa9\xe6\xb0\xa3\xe5\xa5\xbd'],
        [b'\xe5\xa4\xa9\xe6\xb0\xa3\xe5\xb7\xae'],
        [b'\xe4\xb8\x8b\xe9\x9b\xa8'],
        [b'\xe6\x99\xb4\xe5\xa4\xa9'],
        [b'\xe9\x9b\xa8'],
        [b'\xe6\x99\xb4']], dtype=object)>,
 <tf.Tensor: shape=(6, 2), dtype=float32, numpy=
 array([[0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.]], dtype=float32)>)

In [22]:
bert = hub.KerasLayer(
            'https://code.aliyun.com/qhduan/bert_v4/raw/500019068f2c715d4b344c3e2216cef280a7f800/albert_large.tar.gz', 
            trainable=True,
            output_key='pooled_output'
            )

In [23]:
inputs = tf.keras.layers.Input(shape=(None,), dtype=tf.string)

In [24]:
m = inputs

In [25]:
m = bert(inputs)

In [26]:
m = tf.keras.layers.Masking()(m)

In [27]:
#m = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, ))(m)

In [28]:
m = tf.keras.layers.Dense(2, activation='softmax')(m)

In [29]:
model = tf.keras.Model(inputs=inputs, outputs=m)

In [30]:
model.compile(loss='categorical_crossentropy')

In [31]:
model.fit(tx, ty, epochs=10, batch_size=2)

Epoch 1/10
3/3 [==============================] - 23s 799ms/step - loss: 2.7152
Epoch 2/10
3/3 [==============================] - 2s 724ms/step - loss: 9.1883
Epoch 3/10
3/3 [==============================] - 2s 770ms/step - loss: 10.8071
Epoch 4/10
3/3 [==============================] - 2s 764ms/step - loss: 1.0231
Epoch 5/10
3/3 [==============================] - 2s 755ms/step - loss: 1.6644
Epoch 6/10
3/3 [==============================] - 2s 796ms/step - loss: 1.0382
Epoch 7/10
3/3 [==============================] - 2s 817ms/step - loss: 2.4518
Epoch 8/10
3/3 [==============================] - 2s 759ms/step - loss: 1.6124
Epoch 9/10
3/3 [==============================] - 2s 795ms/step - loss: 1.4322
Epoch 10/10
3/3 [==============================] - 2s 802ms/step - loss: 0.6039


In [32]:
logits = model.predict(tx)

In [33]:
pred = logits.argmax(-1).tolist()

In [34]:
pred, y

([0, 0, 0, 0, 0, 0], [1, 0, 0, 1, 0, 1])